In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import hanlp

# 📌 1. HanLP 토큰화 모델 로드
segmenter = hanlp.load("CTB9_TOK_ELECTRA_BASE_CRF")

# 📌 2. 불용어 리스트 정의
chinese_stopwords = set([
    "的", "了", "在", "是", "和", "与", "也", "有", "对", "就", "以", "将", "要",
    "但", "其", "而", "此", "我们", "他们", "你们", "可以", "但是", "这样", "这个",
    "其中", "其中之一", "包括", "根据", "由于", "通过", "此外", "同时",
    "记者", "新闻", "媒体", "采访", "发表", "宣布", "透露", "介绍", "报道称",
    "指出", "强调", "证实", "承认", "认为", "评价", "评论", "提到", "解释", "分析",
    "总结", "预测", "预计", "关注", "反映", "说明", "进一步",
    "今天", "昨天", "明天", "日前", "近日", "本周", "上周", "下周", "目前",
    "现在", "过去", "未来", "今年", "去年", "明年", "此前", "随后", "当地",
    "政府", "机构", "相关", "部门", "官员", "领导人", "代表", "发言人", "部长", "事务", "委员会",
    "一", "二", "三", "这", "不", "可能", "都", "才", "可", "一直", "到", "如果", "将",
    "带来", "十分", "称", "个", "月", "次", "因", "因此", "认为", "如果", "最", "即",
    "当天", "因为", "曾", "号", "第", "等", "相当", "个", "两", "很", "所以", "各种",
    "从而", "仍", "以", "为了", "以及", "据", "并", "过", "几", "号", "立即", "相关",
    "等", "着", "于", "为", "说", "却", "使", "国", "还", "带来", "来说", "而", "为",
    "至", "可以", "很", "会", "除了", "被", "外", "也", "若", "更", "已经", "已", "称",
    "大大", "不仅", "能够", "近日", "并", "再", "一些", "明确", "作为", "向", "于",
    "表示", "以及"
])


C:\Users\82104\IdeaProjects\sentiZh\ZhiTopic\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 📌 3. JSON 데이터 로드
file_path = "../data/final_0307.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 📌 4. 데이터프레임 변환
df = pd.DataFrame(data)

# 📌 5. HanLP로 토큰화 및 불용어 제거
def tokenize_and_filter(text):
    tokens = segmenter(text)
    return " ".join([word for word in tokens if word not in chinese_stopwords])

df["tokenized_content"] = df["cleaned_content"].apply(tokenize_and_filter)


In [11]:
# 📌 6. TF-IDF 벡터라이저 적용
vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(df["tokenized_content"])

# 📌 7. 단어 리스트 가져오기
feature_names = vectorizer.get_feature_names_out()

# 📌 8. 각 기사별 상위 3개 중요 키워드 추출 함수
def get_top_keywords(row, top_n=20):
    indices = np.argsort(row)[::-1]  # 중요도 높은 순 정렬
    top_words = [feature_names[i] for i in indices[:top_n]]
    return ", ".join(top_words)

# 📌 9. 기사별 키워드 추출 및 저장
df["keywords"] = [get_top_keywords(row.toarray().flatten()) for row in tfidf_matrix]

# 📌 11. 결과 확인 (상위 10개 기사)
print("\n✅ 기사별 TF-IDF 키워드 상위 10개:")
print(df[["cleaned_content", "tokenized_content", "keywords"]].head(10))



✅ 기사별 TF-IDF 키워드 상위 10개:
                                     cleaned_content  \
0  7月5日朝鲜连续发射7枚导弹引起国际社会震惊。近日美日在安理会积极推动制裁朝鲜的议案并加紧协...   
1  被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化进...   
2  神秘这是朝鲜留给外界的印象。 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机甚至...   
3  改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶化...   
4  新华网首尔3月7日电 记者李拯宇 干玉兰 美国朝鲜问题特使斯蒂芬博斯沃思7日在韩国说美国愿意...   
5  环球时报驻韩国特约记者沈松报道据韩联社报道朝鲜人民军总参谋部2日对于韩美日3国就朝鲜远程火箭...   
6  中新网4月4日电 综合媒体报道4月4日朝鲜周边紧张态势骤然加剧朝鲜官方已表示即将发射卫星日韩...   
7  新华网北京４月１４日电 综合新华社驻外记者报道联合国安理会１３日就朝鲜发射问题通过主席声明后...   
8  新华网平壤４月１８日电记者高浩荣张滨阳朝鲜人民军总参谋部发言人１８日在平壤发表谈话说美国日本...   
9  中国国产航母及舰载机想像图 澳大利亚悉尼先驱晨报4月29日文章原题我们的问题中国是敌是友？ ...   

                                   tokenized_content  \
0  7月 5日 朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 积极...   
1  外界 视为 破冰 之 旅 朝鲜 副外相 金桂冠 3月 5 6日 纽约 美国 助理 国务卿 希...   
2  神秘 朝鲜 留给 外界 印象 。 正 神秘 国家 情报 费尽心机 投放 卫星 派遣 侦察机 ...   
3  改进型 大浦洞 2 据说 攻击 美 本土 甚至 具有 搭载 核 弹头 技术 朝鲜 最近 频繁...   
4  新华网 首尔 3月 7日 电 李拯宇 干玉兰 美国 朝鲜 问题 特使 斯蒂芬博斯沃思 7日 ...   
5  环球 时报 驻 韩国 特约 沈松 报

In [12]:
output_file = "../data/news_with_tokenized_keywords.json"

# 📌 필요 없는 컬럼 리스트
columns_to_drop = ["sentences", "filtered_sentences", "num_filtered_sentences", "num_sentences", "ratio"]

# 📌 해당 컬럼 삭제 (존재하지 않아도 에러 발생하지 않도록 `errors="ignore"` 설정)
df.drop(columns=columns_to_drop, inplace=True, errors="ignore")

# 📌 2. JSON 덤프를 위한 데이터 변환
json_data = df.to_dict(orient="records")  # DataFrame → JSON 리스트 변환

# 📌 3. Pretty Print JSON 파일 저장
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=4, ensure_ascii=False)